In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd

###########################
# Load CSV Instead of MongoDB
###########################
df = pd.read_csv("aac_shelter_outcomes (3).csv")

# Drop _id if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

###########################
# Branding (Logo + Name)
###########################
image_filename = "Grazioso Salvare Logo.png"
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

###########################
# Dashboard Layout
###########################
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     style={'width': '200px', 'padding': '10px'}),
            href='https://www.snhu.edu',
            target='_blank'
        ),
        html.Div("Created by Bruce Gaudet", style={"fontWeight": "bold", "fontSize": "16px"})
    ], style={'textAlign': 'center'}),

    html.Center(html.B(html.H1('CS-340 Grazioso Salvare Dashboard'))),
    html.Hr(),

    html.Div([
        html.Label("Filter by Rescue Type", style={'fontWeight': 'bold'}),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block'}
        )
    ], style={'width': '25%', 'margin': 'auto'}),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
        page_size=10,
        style_cell={'textAlign': 'left'},
        style_data={'whiteSpace': 'normal'},
        style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'}
    ),

    html.Br(),
    html.Hr(),

    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

###########################
# Callbacks
###########################

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water':
        filtered_df = df[
            (df['breed'].isin(["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"])) &
            (df['sex_upon_outcome'] == "Intact Female") &
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
    elif filter_type == 'mountain':
        filtered_df = df[
            (df['breed'].isin(["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"])) &
            (df['sex_upon_outcome'] == "Intact Male") &
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
    elif filter_type == 'disaster':
        filtered_df = df[
            (df['breed'].isin(["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"])) &
            (df['sex_upon_outcome'] == "Intact Male") &
            (df['age_upon_outcome_in_weeks'].between(20, 300))
        ]
    else:
        filtered_df = df.copy()

    return filtered_df.to_dict('records')


@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div(["No data to display."])
    dff = pd.DataFrame(viewData)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return dcc.Graph(figure=fig)


@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(Output('map-id', "children"),
              [Input('datatable-id', "derived_virtual_data"),
               Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or index is None:
        return []

    dff = pd.DataFrame(viewData)
    row = index[0] if index else 0

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'])
                ])
            ])
        ])
    ]


###########################
# Run App
###########################
app.run_server(debug=True, port=8051)


Dash app running on http://127.0.0.1:8051/
